In [1]:
#include("../../src/repeat_testing_scenarios.jl")
include("../../src/repeat_testing_SC_with_shifts.jl")
using DataFrames
using StatsPlots
using CSV

#model options
VL_model = kissler_model_no 
Inf_model = flat_inf_model_no
onset_opt = PVT_model_opt
peak_inf_opt = unif_peakinf_opt
PCR_sens_max = 0.83 

0.83

In [2]:
Ntot = 10000
Nscens = length(scen_names)
Adhs = collect(0.1:0.1:1.0)
Nadhs = length(Adhs)
Confs = [true, false]
Nconfs = 2
TATorig = mean(PCRTATdist)
TATscales = [24.0, 36.0, 48.0, 60.0, 72.0] ./ TATorig
Day5release = true

true

Debugging zone

In [3]:
# SDS = rand(1:14)
# print(SDS,'\n')
# TD, TT, RD = scenario_3_setup(21, SDS)
# print(TD[TT .== 1.0],' ', TD[TT .== 0.0],'\n')
# print(RD[TT .== 1.0],' ', RD[TT .== 0.0],'\n')
# Nrepeats = Int64(ceil((21 - 13 + SDS)/14))
# shifts = vcat(shift_pattern[SDS:end], repeat(shift_pattern,Nrepeats))
# shift_days = collect(1:length(shifts))[shifts]
# print(shift_days,'\n')

In [4]:
# Nscens = length(scen_names)

# simtest = get_no_testing_scenario(2, 1.0; Day5release_bool =  Day5release)

# simscens, sim_names = run_testing_scenarios_vs_baseline(simtest, 0.7, false; Day5release_bool = Day5release)
# keys_loop = ["inf_profile_isolation", "isol_days","symp_day"]

# for i in 1:2
#     print(i,'\n')
#     for key in keys_loop
#         if haskey(simtest,key)
#             print(key, ' ', simtest[key][i],'\n')
#         end
#     end
# end



# for j in [9,1,2,3,4,5,6,7,8]
#     print(sim_names[j],'\n')
#     for i in 1:2
#         print(i,'\n')
#         for key in keys_loop
#             print(key, ' ', simscens[j][key][i],'\n')
#         end
#     end
# end
#something weird going on here with symp isol days in no testing case only

In [ ]:
LFD_model = SC_data_2021
sim_baseline = get_no_testing_scenario(Ntot, 1.0; Day5release_bool = Day5release)
start = true
df = []
for c in Confs
    print("Conf ",c,'\n')
    for a in Adhs
        print("Adh ",a,'\n')
        PCR_TaT_scale = 1.0
        scenarios, names = run_testing_scenarios_vs_baseline(sim_baseline, a, c; 
                           Day5release_bool = Day5release)
        dfh = DataFrame(Scenario=repeat(names,Ntot),WorkInfDays=zeros(Nscens*Ntot),
                    WorkInfPot=zeros(Nscens*Ntot), ConfPCR=fill(c,Nscens*Ntot),
                    Adherence=fill(a,Nscens*Ntot), MeanTaT=fill(TATorig,Nscens*Ntot), 
                    WorkIsolDays=zeros(Nscens*Ntot))
        for i in 1:Ntot
            for j in 1:Nscens
                dfh["WorkInfDays"][Nscens*(i-1)+j] = scenarios[j]["inf_days"][i]
                dfh["WorkInfPot"][Nscens*(i-1)+j] = sum(scenarios[j]["inf_profile_isolation"][i])
                dfh["WorkIsolDays"][Nscens*(i-1)+j] = scenarios[j]["tot_work_isol_days"][i]
            end
        end
        if start
            df = dfh
        else
            df = vcat(df,dfh)
        end
        start = false
    end
    
    for ts in TATscales
        PCR_TaT_scale = ts
        scenarios, names = run_testing_scenarios_vs_baseline(sim_baseline, 0.7, c; 
                           Day5release_bool = Day5release)
        dfh = DataFrame(Scenario=repeat(names,Ntot),WorkInfDays=zeros(Nscens*Ntot),
                    WorkInfPot=zeros(Nscens*Ntot),
                    ConfPCR=fill(c,Nscens*Ntot),Adherence=fill(0.7,Nscens*Ntot),
                    MeanTaT=fill(TATorig*ts,Nscens*Ntot), WorkIsolDays=zeros(Nscens*Ntot))
        for i in 1:Ntot
            for j in 1:Nscens
                dfh["WorkInfDays"][Nscens*(i-1)+j] = scenarios[j]["inf_days"][i]
                dfh["WorkInfPot"][Nscens*(i-1)+j] = sum(scenarios[j]["inf_profile_isolation"][i])
                dfh["WorkIsolDays"][Nscens*(i-1)+j] = scenarios[j]["tot_work_isol_days"][i]
            end
        end
        if start
            df = dfh
        else
            df = vcat(df,dfh)
        end
    end
    PCR_TaT_scale = 1.0
end
CSV.write("Psymp50_Pisol100_PCRmax83_with_shifts.csv",df)

In [6]:
LFD_model = SC_data_2021
sim_baseline = get_no_testing_scenario(Ntot, 1.0; Day5release_bool = Day5release)
start = true
df = []
for c in Confs
    for a in Adhs
        PCR_TaT_scale = 1.0
        scenarios, names = run_testing_scenarios_vs_baseline(sim_baseline, a, c;
                                  LFD_AllorNone = true, Day5release_bool = Day5release)
        dfh = DataFrame(Scenario=repeat(names,Ntot),WorkInfDays=zeros(Nscens*Ntot),
                    WorkInfPot=zeros(Nscens*Ntot), ConfPCR=fill(c,Nscens*Ntot),
                    Adherence=fill(a,Nscens*Ntot), MeanTaT=fill(TATorig,Nscens*Ntot), 
                    WorkIsolDays=zeros(Nscens*Ntot))
        for i in 1:Ntot
            for j in 1:Nscens
                dfh["WorkInfDays"][Nscens*(i-1)+j] = scenarios[j]["inf_days"][i]
                dfh["WorkInfPot"][Nscens*(i-1)+j] = sum(scenarios[j]["inf_profile_isolation"][i])
                dfh["WorkIsolDays"][Nscens*(i-1)+j] = scenarios[j]["tot_work_isol_days"][i]
            end
        end
        if start
            df = dfh
        else
            df = vcat(df,dfh)
        end
        start = false
    end
    
    for ts in TATscales
        PCR_TaT_scale = ts
        scenarios, names = run_testing_scenarios_vs_baseline(sim_baseline, 0.7, c;
                             LFD_AllorNone = true, Day5release_bool = Day5release)
        dfh = DataFrame(Scenario=repeat(names,Ntot),WorkInfDays=zeros(Nscens*Ntot),
                    WorkInfPot=zeros(Nscens*Ntot),
                    ConfPCR=fill(c,Nscens*Ntot),Adherence=fill(0.7,Nscens*Ntot),
                    MeanTaT=fill(TATorig*ts,Nscens*Ntot), WorkIsolDays=zeros(Nscens*Ntot))
        for i in 1:Ntot
            for j in 1:Nscens
                dfh["WorkInfDays"][Nscens*(i-1)+j] = scenarios[j]["inf_days"][i]
                dfh["WorkInfPot"][Nscens*(i-1)+j] = sum(scenarios[j]["inf_profile_isolation"][i])
                dfh["WorkIsolDays"][Nscens*(i-1)+j] = scenarios[j]["tot_work_isol_days"][i]
            end
        end
        if start
            df = dfh
        else
            df = vcat(df,dfh)
        end
    end
    PCR_TaT_scale = 1.0
end
CSV.write("Psymp50_Pisol100_PCRmax83_AoNcomply_with_shifts.csv",df)

"Psymp50_Pisol100_PCRmax83_AoNcomply_with_shifts.csv"

In [7]:
for i in 1:Nscens
    print(mean(df[df["Scenario"] .== scen_names[i],"WorkInfPot"]),'\n')
end

0.5500080756063414
0.6852007361578233
0.693593907431313
0.7340124190243643
0.787273527521777
0.6511782332893773
0.4737189874164037
0.523836440191847
1.1186208062713303


In [5]:
LFD_model = SC_data_Oct

sim_baseline = get_no_testing_scenario(Ntot, 1.0; Day5release_bool = Day5release)
start = true
df = []
for c in Confs
    print("Conf ",c,'\n')
    for a in Adhs
        print("Adh ",a,'\n')
        PCR_TaT_scale = 1.0
        scenarios, names = run_testing_scenarios_vs_baseline(sim_baseline, a, c; 
                           Day5release_bool = Day5release)
        dfh = DataFrame(Scenario=repeat(names,Ntot),WorkInfDays=zeros(Nscens*Ntot),
                    WorkInfPot=zeros(Nscens*Ntot), ConfPCR=fill(c,Nscens*Ntot),
                    Adherence=fill(a,Nscens*Ntot), MeanTaT=fill(TATorig,Nscens*Ntot), 
                    WorkIsolDays=zeros(Nscens*Ntot))
        for i in 1:Ntot
            for j in 1:Nscens
                dfh["WorkInfDays"][Nscens*(i-1)+j] = scenarios[j]["inf_days"][i]
                dfh["WorkInfPot"][Nscens*(i-1)+j] = sum(scenarios[j]["inf_profile_isolation"][i])
                dfh["WorkIsolDays"][Nscens*(i-1)+j] = scenarios[j]["tot_work_isol_days"][i]
            end
        end
        if start
            df = dfh
        else
            df = vcat(df,dfh)
        end
        start = false
    end
    
    for ts in TATscales
        PCR_TaT_scale = ts
        scenarios, names = run_testing_scenarios_vs_baseline(sim_baseline, 0.7, c; 
                           Day5release_bool = Day5release)
        dfh = DataFrame(Scenario=repeat(names,Ntot),WorkInfDays=zeros(Nscens*Ntot),
                    WorkInfPot=zeros(Nscens*Ntot),
                    ConfPCR=fill(c,Nscens*Ntot),Adherence=fill(0.7,Nscens*Ntot),
                    MeanTaT=fill(TATorig*ts,Nscens*Ntot), WorkIsolDays=zeros(Nscens*Ntot))
        for i in 1:Ntot
            for j in 1:Nscens
                dfh["WorkInfDays"][Nscens*(i-1)+j] = scenarios[j]["inf_days"][i]
                dfh["WorkInfPot"][Nscens*(i-1)+j] = sum(scenarios[j]["inf_profile_isolation"][i])
                dfh["WorkIsolDays"][Nscens*(i-1)+j] = scenarios[j]["tot_work_isol_days"][i]
            end
        end
        if start
            df = dfh
        else
            df = vcat(df,dfh)
        end
    end
    PCR_TaT_scale = 1.0
end
CSV.write("Psymp50_Pisol100_PCRmax83_lowsens_day5release_withshifts.csv",df)

Conf true
Adh 0.1
Adh 0.2
Adh 0.3
Adh 0.4
Adh 0.5
Adh 0.6
Adh 0.7
Adh 0.8
Adh 0.9
Adh 1.0
Conf false
Adh 0.1
Adh 0.2
Adh 0.3
Adh 0.4
Adh 0.5
Adh 0.6
Adh 0.7
Adh 0.8
Adh 0.9
Adh 1.0


"Psymp50_Pisol100_PCRmax83_lowsens_day5release_withshifts.csv"

In [6]:
LFD_model = SC_data_Oct
sim_baseline = get_no_testing_scenario(Ntot, 1.0; Day5release_bool = Day5release)
start = true
df = []
for c in Confs
    for a in Adhs
        PCR_TaT_scale = 1.0
        scenarios, names = run_testing_scenarios_vs_baseline(sim_baseline, a, c;
                                  LFD_AllorNone = true, Day5release_bool = Day5release)
        dfh = DataFrame(Scenario=repeat(names,Ntot),WorkInfDays=zeros(Nscens*Ntot),
                    WorkInfPot=zeros(Nscens*Ntot), ConfPCR=fill(c,Nscens*Ntot),
                    Adherence=fill(a,Nscens*Ntot), MeanTaT=fill(TATorig,Nscens*Ntot), 
                    WorkIsolDays=zeros(Nscens*Ntot))
        for i in 1:Ntot
            for j in 1:Nscens
                dfh["WorkInfDays"][Nscens*(i-1)+j] = scenarios[j]["inf_days"][i]
                dfh["WorkInfPot"][Nscens*(i-1)+j] = sum(scenarios[j]["inf_profile_isolation"][i])
                dfh["WorkIsolDays"][Nscens*(i-1)+j] = scenarios[j]["tot_work_isol_days"][i]
            end
        end
        if start
            df = dfh
        else
            df = vcat(df,dfh)
        end
        start = false
    end
    
    for ts in TATscales
        PCR_TaT_scale = ts
        scenarios, names = run_testing_scenarios_vs_baseline(sim_baseline, 0.7, c;
                             LFD_AllorNone = true, Day5release_bool = Day5release)
        dfh = DataFrame(Scenario=repeat(names,Ntot),WorkInfDays=zeros(Nscens*Ntot),
                    WorkInfPot=zeros(Nscens*Ntot),
                    ConfPCR=fill(c,Nscens*Ntot),Adherence=fill(0.7,Nscens*Ntot),
                    MeanTaT=fill(TATorig*ts,Nscens*Ntot), WorkIsolDays=zeros(Nscens*Ntot))
        for i in 1:Ntot
            for j in 1:Nscens
                dfh["WorkInfDays"][Nscens*(i-1)+j] = scenarios[j]["inf_days"][i]
                dfh["WorkInfPot"][Nscens*(i-1)+j] = sum(scenarios[j]["inf_profile_isolation"][i])
                dfh["WorkIsolDays"][Nscens*(i-1)+j] = scenarios[j]["tot_work_isol_days"][i]
            end
        end
        if start
            df = dfh
        else
            df = vcat(df,dfh)
        end
    end
    PCR_TaT_scale = 1.0
end
CSV.write("Psymp50_Pisol100_PCRmax83_AoNcomply_lowsens_day5release_with_shifts.csv",df)

"Psymp50_Pisol100_PCRmax83_AoNcomply_lowsens_day5release_with_shifts.csv"

In [3]:
print(PCR_VL0,'\n',PCR_VLdisp,'\n')

In [9]:
PCR_TaT_scale = 1.0
scenarios, names = run_testing_scenarios_vs_baseline(sim_baseline, 0.7, false)
inf_len = max(length.(scenarios[Nscens]["infection_profiles"])...)
VL_len = max(length.(scenarios[Nscens]["VL_profiles"])...)
all_infs = zeros(Float64,(Ntot,inf_len))
all_symp_infs = zeros(Float64,(Ntot,inf_len))
all_sQ_infs = zeros(Float64,(Ntot,inf_len))
all_2LFD_infs = zeros(Float64,(Ntot,inf_len))
all_DailyLFD_infs = zeros(Float64,(Ntot,inf_len))
all_PCR = zeros(Float64,(Ntot,VL_len))
all_LFD = zeros(Float64,(Ntot,VL_len))
all_LFD2 = zeros(Float64,(Ntot,VL_len))
for i in 1:Ntot
    all_infs[i,1:length(scenarios[Nscens]["infection_profiles"][i])] = 
                      scenarios[Nscens]["infection_profiles"][i]
    all_symp_infs[i,1:length(scenarios[Nscens]["inf_profile_isolation"][i])] = 
                      scenarios[Nscens]["inf_profile_isolation"][i]
    all_sQ_infs[i,1:length(scenarios[1]["inf_profile_isolation"][i])] = 
                      scenarios[1]["inf_profile_isolation"][i]
    all_2LFD_infs[i,1:length(scenarios[5]["inf_profile_isolation"][i])] = 
                      scenarios[5]["inf_profile_isolation"][i]
    all_DailyLFD_infs[i,1:length(scenarios[6]["inf_profile_isolation"][i])] = 
                      scenarios[6]["inf_profile_isolation"][i]
    all_PCR[i,1:length(scenarios[Nscens]["VL_profiles"][i])] = 
                      PCRtest_positive_prob.(scenarios[Nscens-1]["VL_profiles"][i])
    all_LFD[i,1:length(scenarios[Nscens]["VL_profiles"][i])] = 
                      LFDtest_positive_prob.(scenarios[7]["VL_profiles"][i])
end

In [45]:
plot(0:(inf_len-1),transpose(mean(all_infs,dims=1)),linewidth=2,label="Infectiousness")
plot!(0:(inf_len-1),transpose(mean(all_symp_infs,dims=1)),linewidth=2,label="With Symp. Isol")
plot!(0:(inf_len-1),transpose(mean(all_2LFD_infs,dims=1)),linewidth=2, label="2 LFDs only", xlim=(0,30),ylim=(0,0.5))
plot!(0:(inf_len-1),transpose(mean(all_sQ_infs,dims=1)),linewidth=2, label="2 LFDs + 1 PCR")
plot!(0:(inf_len-1),transpose(mean(all_DailyLFD_infs,dims=1)), linewidth=2,label="Daily LFDs only", 
    xlim=(0,30),ylim=(0,0.5),size=(500,300),
    xlabel="Days since infection",ylabel="Mean infectiousness (a.u.)", leftmargin=5Plots.mm, 
    bottommargin=5Plots.mm)

savefig("Infectiousness_time.png")

In [44]:
plot(0:(VL_len-1),transpose(mean(all_PCR,dims=1)),label="PCR",linewidth=2)
plot!(0:(VL_len-1),transpose(mean(all_LFD,dims=1)),label="LFD",
    xlim=(0,30),ylim=(0,1),linewidth=2,size=(400,250),
    xlabel="Days since infection",ylabel="Mean test positive probability",
    leftmargin=5Plots.mm, bottommargin=5Plots.mm)
savefig("Testpos_time.png")

In [47]:
x = 0:0.1:12
y1 = PCRtest_positive_prob.(x)
y2 = LFDtest_positive_prob.(x)
plot(x,y1,label="PCR",linewidth=2)
plot!(x,y2,label="LFD",linewidth=2, size=(400,250),xlabel="Viral load (log10 copies/ml)",
    ylabel="Test postive probability", leftmargin=5Plots.mm, bottommargin=5Plots.mm,legend=:bottomright)
savefig("Testpos.png")

In [46]:
x = 0:1.0:144
y = pdf(Gamma(6.75,20/3),x)
plot(x,y,color=:black,linewidth=3, label="Original (45h)")

PCR_TaT_scale = TATscales[1]
y = pdf(Gamma(6.75,TATscales[1]*20/3),x)
plot!(x,y,color=:green,style=:dashdot,linewidth=2,label="24h mean delay")
PCR_TaT_scale = TATscales[5]
y = pdf(Gamma(6.75,TATscales[5]*20/3),x)
plot!(x,y,xlim=(0,144),color=:red,style=:dot,linewidth=2,label="72h mean delay",
    size=(400,250),xlabel="PCR turnaround time",ylabel="Probability density",
    leftmargin=5Plots.mm, bottommargin=5Plots.mm)
savefig("PCRturnaround.png")